In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/huggingface/transformers@main
!pip install causal-conv1d>=1.2.0
!pip install mamba-ssm

In [2]:
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("tri-ml/mamba-7b-rw", torch_dtype=torch.float16)
model = AutoModelForCausalLM.from_pretrained("tri-ml/mamba-7b-rw", torch_dtype=torch.float16)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/5.21k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/28.6G [00:00<?, ?B/s]

The fast path is not available because on of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the naive implementation. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d
Some weights of the model checkpoint at tri-ml/mamba-7b-rw were not used when initializing MambaForCausalLM: ['model.lm_head.weight']
- This IS expected if you are initializing MambaForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MambaForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


generation_config.json:   0%|          | 0.00/148 [00:00<?, ?B/s]

In [3]:
model = model.to('cuda:0')
model.generation_config.pad_token_id = model.generation_config.eos_token_id

In [4]:
def process_prompts(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as file:
        prompts = file.read().split('\n\n')

    with open(output_file, 'w', encoding='utf-8') as outfile:
        for i, prompt in enumerate(prompts):
            if prompt.strip():
                input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
                outputs = model.generate(
                    input_ids["input_ids"],
                    max_new_tokens=64,
                    early_stopping=True,
                    num_return_sequences=1
                )
                response = tokenizer.decode(outputs[0])

                # response = response.split('The correct answer is:')[1].strip()
                response = response.split('Classify the sentiment of the following statement:')[1].strip()
                question = prompt.split('---')[1]
                outfile.write(f"Question {i+1}: {question}\n\nModel's response for question {i+1}: {response}\n------------------------------------------\n")
                print(f"Processed prompt {i+1}")
                print("\n")
                print(f"Question: {question}")
                print("\n")
                print(f"Response: {response}")
                print("---------------------------------")

    print("All responses have been successfully processed and saved.")

In [5]:
input_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/sentiment_prompts.txt'
output_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/Responses/mamba_7b_responses_for_sentiment.txt'

process_prompts(input_file, output_file)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Processed prompt 1


Question: 
Classify the sentiment of the following statement:
Statement: The sunset over the ocean was breathtaking. Sentiment:


Response: Statement: The sunset over the ocean was breathtaking. Sentiment: Positive.
---
---------------------------------
Processed prompt 2


Question: 
Classify the sentiment of the following statement:
Statement: The coffee machine broke down again. Sentiment:


Response: Statement: The coffee machine broke down again. Sentiment: Negative.
---
---------------------------------
Processed prompt 3


Question: 
Classify the sentiment of the following statement:
Statement: Our meeting is scheduled for tomorrow. Sentiment:


Response: Statement: Our meeting is scheduled for tomorrow. Sentiment: Positive.
---
---------------------------------
Processed prompt 4


Question: 
Classify the sentiment of the following statement:
Statement: I am so proud of my team's hard work. Sentiment:


Response: Statement: I am so proud of my team's hard w

In [6]:
input_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/sentiment_prompts_random_label.txt'
output_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/Responses/mamba_7b_responses_for_sentiment_random_label.txt'

process_prompts(input_file, output_file)

Processed prompt 1


Question: 
Classify the sentiment of the following statement:
Statement: The sunset over the ocean was breathtaking. Sentiment:


Response: Statement: The sunset over the ocean was breathtaking. Sentiment: Positive.
---
---------------------------------
Processed prompt 2


Question: 
Classify the sentiment of the following statement:
Statement: The coffee machine broke down again. Sentiment:


Response: Statement: The coffee machine broke down again. Sentiment: Negative.
---
---------------------------------
Processed prompt 3


Question: 
Classify the sentiment of the following statement:
Statement: Our meeting is scheduled for tomorrow. Sentiment:


Response: Statement: Our meeting is scheduled for tomorrow. Sentiment: Positive.
---
---------------------------------
Processed prompt 4


Question: 
Classify the sentiment of the following statement:
Statement: I am so proud of my team's hard work. Sentiment:


Response: Statement: I am so proud of my team's hard w

In [7]:
input_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/sentiment_prompts_no_demos.txt'
output_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/Responses/mamba_7b_responses_for_sentiment_no_demos.txt'

process_prompts(input_file, output_file)

Processed prompt 1


Question: 
Classify the sentiment of the following statement:
Statement: The sunset over the ocean was breathtaking. Sentiment:


Response: Statement: The sunset over the ocean was breathtaking. Sentiment: Positive
---------------------------------
Processed prompt 2


Question: 
Classify the sentiment of the following statement:
Statement: The coffee machine broke down again. Sentiment:


Response: Statement: The coffee machine broke down again. Sentiment: Negative
---
---------------------------------
Processed prompt 3


Question: 
Classify the sentiment of the following statement:
Statement: Our meeting is scheduled for tomorrow. Sentiment:


Response: Statement: Our meeting is scheduled for tomorrow. Sentiment: Positive
---------------------------------
Processed prompt 4


Question: 
Classify the sentiment of the following statement:
Statement: I am so proud of my team's hard work. Sentiment:


Response: Statement: I am so proud of my team's hard work. Sentim

In [8]:
input_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/sentiment_prompts_cot.txt'
output_file = '/content/drive/MyDrive/NLP Project/Data/Sentiment/Responses/mamba_7b_responses_for_sentiment_cot.txt'

process_prompts(input_file, output_file)

Processed prompt 1


Question: 
Classify the sentiment of the following statement:
Statement: The sunset over the ocean was breathtaking. Sentiment:


Response: Statement: The sunset over the ocean was breathtaking. Sentiment: Positive.
Reason: The statement describes an enjoyable experience.
---
---------------------------------
Processed prompt 2


Question: 
Classify the sentiment of the following statement:
Statement: The coffee machine broke down again. Sentiment:


Response: Statement: The coffee machine broke down again. Sentiment: Negative.
Reason: The statement expresses frustration about the coffee machine.
---
---------------------------------
Processed prompt 3


Question: 
Classify the sentiment of the following statement:
Statement: Our meeting is scheduled for tomorrow. Sentiment:


Response: Statement: Our meeting is scheduled for tomorrow. Sentiment: Positive.
Reason: The statement conveys a positive expectation about the meeting.
---
---------------------------------


In [9]:
def process_prompts(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as file:
        prompts = file.read().split('\n\n')

    with open(output_file, 'w', encoding='utf-8') as outfile:
        for i, prompt in enumerate(prompts):
            if prompt.strip():
                input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
                outputs = model.generate(
                    input_ids["input_ids"],
                    max_new_tokens=64,
                    early_stopping=True,
                    num_return_sequences=1
                )
                response = tokenizer.decode(outputs[0])

                response = response.split('The correct answer is:')[1].strip()
                question = prompt.split('---')[1]
                outfile.write(f"Question {i+1}: {question}\n\nModel's response for question {i+1}: {response}\n------------------------------------------\n")
                print(f"Processed prompt {i+1}")
                print("\n")
                print(f"Question: {question}")
                print("\n")
                print(f"Response: {response}")
                print("---------------------------------")

    print("All responses have been successfully processed and saved.")

In [10]:
input_file = '/content/drive/MyDrive/NLP Project/Data/random_arithmetic_prompts.txt'
output_file = '/content/drive/MyDrive/NLP Project/Data/mamba_7b_responses_for_random_arithmetic.txt'

process_prompts(input_file, output_file)

Processed prompt 1


Question: 
Calculate the next problem and provide the answer:
Calculate 65 + 60? The correct answer is:


Response: 65 + 60 = 125
---
Calculate the next problem and provide the answer:
Calculate the sum of the following numbers:
1. Calculate the sum of the following numbers:
2. Calculate the sum of the following numbers:
3. Calculate the sum of the following numbers:
4. Calculate
---------------------------------
Processed prompt 2


Question: 
Calculate the next problem and provide the answer:
Calculate 90 + 23? The correct answer is:


Response: 23 + 23 = 46
46 + 23 = 69
69 + 23 = 92
92 + 23 = 115
115 + 23 = 138
138 + 23 = 161
161 + 23 = 184
184 + 23 = 207
207 + 23 = 230
230 + 23 = 253
253 + 23
---------------------------------
Processed prompt 3


Question: 
Calculate the next problem and provide the answer:
Calculate 57 / 53? The correct answer is:


Response: 57 / 53 = 1.09
---
Calculate the next problem and provide the answer:
Calculate the sum of the followi